In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
import tools_for_tobac as toolstobac

In [3]:
from open_experiment import control_deaccu

In [4]:
path='/scratch/wcq7pz/exp_levante_post/'
## open topography, land_fraction
topo5km = xr.open_dataset(path+'topography_dom03_5km.nc')
topo5km.coords['lon'] = (topo5km.coords['lon'] + 180) % 360 - 180
topo5km  = topo5km.sortby(topo5km.lon)


In [5]:
ds_topo = topo5km['topography_c'].broadcast_like(control_deaccu['time'])


In [6]:
grid_area = xr.open_dataset('gridarea_dom3p5.nc')
grid_area.coords['lon'] = (grid_area.coords['lon'] + 180) % 360 - 180
grid_area = grid_area.sortby(grid_area.lon)

### open OCS masks 

In [7]:
## Save mask of OCS (objects lasting > 3h)  
nMask_c_AB = xr.open_dataset('nMask_control_ocs_AB.nc'); 
nMask_f_AB = xr.open_dataset('nMask_fixedSM_ocs_AB.nc')

nMask_c_SESA = xr.open_dataset('nMask_control_ocs_SESA.nc'); 
nMask_f_SESA = xr.open_dataset('nMask_fixedSM_ocs_SESA.nc')

### prepare topography information

In [9]:
ds_topo = topo5km['topography_c'].broadcast_like(control_deaccu['time'])

In [10]:
# Filter the 'mask' variable to only include the values present in the original dataframe

## Select objects with a lifetime larger than 3 hours
c_topo_AB =  ds_topo.where(ds_topo['time'].isin(nMask_c_AB.time), drop=True)
f_topo_AB =  ds_topo.where(ds_topo['time'].isin(nMask_f_AB.time), drop=True)

c_topo_SESA =  ds_topo.where(ds_topo['time'].isin(nMask_c_SESA.time), drop=True)
f_topo_SESA =  ds_topo.where(ds_topo['time'].isin(nMask_f_SESA.time), drop=True)

### identify objects

In [11]:
grid_area = xr.open_dataset('gridarea_dom3p5.nc')
grid_area.coords['lon'] = (grid_area.coords['lon'] + 180) % 360 - 180
grid_area = grid_area.sortby(grid_area.lon)


In [12]:
## select grid_area corresponding to nMask boundaries
gnpc_AB = grid_area.sel(lon=slice(nMask_c_AB.lon.min(),nMask_c_AB.lon.max()), 
                            lat=slice(nMask_c_AB.lat.min(),nMask_c_AB.lat.max())).cell_area.values / 1e6;
gnpf_AB = grid_area.sel(lon=slice(nMask_f_AB.lon.min(),nMask_f_AB.lon.max()), 
                            lat=slice(nMask_f_AB.lat.min(),nMask_f_AB.lat.max())).cell_area.values / 1e6;


In [14]:
#### Amazon 
arrayTOPO_control_AB = np.squeeze(c_topo_AB.sel(
    lon=slice(nMask_c_AB.lon.min(),nMask_c_AB.lon.max()),lat=slice(nMask_c_AB.lat.min(),nMask_c_AB.lat.max())).values)
arrayTOPO_fixedSM_AB = np.squeeze(f_topo_AB.sel(
    lon=slice(nMask_f_AB.lon.min(),nMask_f_AB.lon.max()),lat=slice(nMask_f_AB.lat.min(),nMask_f_AB.lat.max())).values)

In [15]:
## get object properties

df_topo_c_AB = toolstobac.mask_var(nMask_c_AB.mask_obs,arrayTOPO_control_AB,gnpc_AB,lon1=nMask_c_AB.coords['lon'].values,
                            lat1=nMask_c_AB.coords['lat'].values,rr_limit=0,timeds=pd.Series(nMask_c_AB.time))
df_topo_f_AB = toolstobac.mask_var(nMask_f_AB.mask_obs,arrayTOPO_fixedSM_AB,gnpf_AB,lon1=nMask_f_AB.coords['lon'].values,
                            lat1=nMask_f_AB.coords['lat'].values,rr_limit=0,timeds=pd.Series(nMask_f_AB.time))


In [17]:
###### SESA 

array_sesa = (c_topo_SESA.sel(lon=slice(nMask_c_SESA.lon.min(),nMask_c_SESA.lon.max()),lat=slice(nMask_c_SESA.lat.min(),nMask_c_SESA.lat.max())))
# Check if all grid points for each time step are equal to 0
all_zeros = (array_sesa == 0).all(dim=('lat', 'lon'))
# Replace the grid points with 1 where all values are 0
array_sesa = array_sesa.where(~all_zeros, 1)
arrayTOPO_control_SESA = array_sesa.values

array_fsesa = (f_topo_SESA.sel(lon=slice(nMask_f_SESA.lon.min(),nMask_f_SESA.lon.max()),lat=slice(nMask_f_SESA.lat.min(),nMask_f_SESA.lat.max())))
all_fzeros = (array_fsesa == 0).all(dim=('lat', 'lon'))
array_fsesa = array_fsesa.where(~all_fzeros, 1)
arrayTOPO_fixedSM_SESA = array_fsesa.values

In [23]:
del(array_sesa,array_fsesa,all_zeros,all_fzeros)

In [18]:
gnpc_SESA = grid_area.sel(lon=slice(nMask_c_SESA.lon.min(),nMask_c_SESA.lon.max()), 
                            lat=slice(nMask_c_SESA.lat.min(),nMask_c_SESA.lat.max())).cell_area.values / 1e6;
gnpf_SESA = grid_area.sel(lon=slice(nMask_f_SESA.lon.min(),nMask_f_SESA.lon.max()),
                            lat=slice(nMask_f_SESA.lat.min(),nMask_f_SESA.lat.max())).cell_area.values / 1e6;

In [21]:
## get object properties

df_topo_c_SESA = toolstobac.mask_var(nMask_c_SESA.mask_obs,arrayTOPO_control_SESA,gnpc_SESA,lon1=nMask_c_SESA.coords['lon'].values,
                            lat1=nMask_c_SESA.coords['lat'].values,rr_limit=0,timeds=pd.Series(nMask_c_SESA.time))
df_topo_f_SESA = toolstobac.mask_var(nMask_f_SESA.mask_obs,arrayTOPO_fixedSM_SESA,gnpf_SESA,lon1=nMask_f_SESA.coords['lon'].values,
                            lat1=nMask_f_SESA.coords['lat'].values,rr_limit=0,timeds=pd.Series(nMask_f_SESA.time))
                           

### Add topo info to OCS_dataframe columns 

In [22]:
def add_topo_cols(df,df_topo):
    df['topo_mean'] = pd.to_numeric(df_topo['mean'],errors = 'coerce');
    df['topo_median'] = pd.to_numeric(df_topo['median'],errors = 'coerce');
    df['topo_std'] = pd.to_numeric(df_topo['std'],errors = 'coerce');
    df['topo_max'] = pd.to_numeric(df_topo['max'],errors = 'coerce');
    return(df)

In [23]:
df_stage_c_AB = pd.read_pickle('pkl_files/df_stage_c_gt3h_AB.pkl');
df_stage_f_AB = pd.read_pickle('pkl_files/df_stage_f_gt3h_AB.pkl')

df_stage_c_SESA = pd.read_pickle('pkl_files/df_stage_c_gt3h_SESA.pkl');
df_stage_f_SESA = pd.read_pickle('pkl_files/df_stage_f_gt3h_SESA.pkl')


In [24]:
df_ocs_control_AB = add_topo_cols(df_stage_c_AB,df_topo_c_AB);
df_ocs_fixedSM_AB = add_topo_cols(df_stage_f_AB,df_topo_f_AB);


In [25]:
df_ocs_control_SESA = add_topo_cols(df_stage_c_SESA,df_topo_c_SESA);
df_ocs_fixedSM_SESA = add_topo_cols(df_stage_f_SESA,df_topo_f_SESA);



In [26]:
df_ocs_control_AB.to_pickle('pkl_files/df_ocs_control_AB.pkl'); 
df_ocs_fixedSM_AB.to_pickle('pkl_files/df_ocs_fixedSM_AB.pkl');

df_ocs_control_SESA.to_pickle('pkl_files/df_ocs_control_SESA.pkl'); 
df_ocs_fixedSM_SESA.to_pickle('pkl_files/df_ocs_fixedSM_SESA.pkl'); 

